In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
import sklearn.metrics
import math
import random
from pandas import DataFrame,Series
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time
import matplotlib.pyplot as plt

Importing data from text file:

In [2]:
data = pd.read_csv('LabelledData.txt', sep=",,,", header = None)

/home/vasu/Downloads/Anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [3]:
data.columns = ["rqst",'r_type']
data['r_type']=data['r_type'].str.strip()    # to remove extra spaces

In [4]:
data['r_type'].unique()               #Five categories

array(['unknown', 'what', 'when', 'who', 'affirmation'], dtype=object)

In [5]:
data.head()

,rqst,r_type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [6]:
data.describe()

,rqst,r_type
count,1483,1483
unique,1476,5
top,what is the speed of the mississippi river ?,what
freq,3,609


Note:: Data seems clean :: NO null values or missing 

In [7]:
data.groupby('r_type').count()

,rqst
r_type,
affirmation,104
unknown,272
what,609
when,96
who,402


In [8]:
np.array(data.rqst)      # sample of queostions

array(['how did serfdom develop in and then leave russia ? ',
       'what films featured the character popeye doyle ? ',
       "how can i find a list of celebrities ' real names ? ", ...,
       'does this hose have one ? ', 'can i get it in india ? ',
       'would this work on a 2008 ford edge with a naked roof ? '], dtype=object)

### Using sklean TfidfVectorizer to transform text

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df = 1)
tfidf_matrix = tfidf_vectorizer.fit_transform(data.rqst)


#### Train-test Data slicing

In [10]:
tf_idf_data = tfidf_matrix.todense()
labels = data.r_type
X_train, X_test, y_train, y_test  =   train_test_split(tf_idf_data, labels, test_size=.3)


In [11]:
#analyzing the shape of the training and test data-set:
print('Training Data dimension: '+str(X_train.shape))
print('Test Data dimension: '+str(X_test.shape))

Training Data dimension: (1038, 3636)
Test Data dimension: (445, 3636)


In [12]:
from sklearn.ensemble import RandomForestClassifier
#Training
model=RandomForestClassifier(n_estimators=150)
model=model.fit(X_train,y_train)
##  Predict
predictions=model.predict(X_test)
accuracy=sklearn.metrics.accuracy_score(y_test, predictions)
print "Accuracy :: \t",accuracy

Accuracy :: 	0.961797752809


Confusion Matrix and Report

In [13]:
print(sklearn.metrics.confusion_matrix(y_test,predictions))
print(sklearn.metrics.classification_report(y_test,predictions))

[[ 24   2   1   0   0]
 [  0  73   3   1   1]
 [  0   0 175   2   2]
 [  0   1   4  23   0]
 [  0   0   0   0 133]]
             precision    recall  f1-score   support

affirmation       1.00      0.89      0.94        27
    unknown       0.96      0.94      0.95        78
       what       0.96      0.98      0.97       179
       when       0.88      0.82      0.85        28
        who       0.98      1.00      0.99       133

avg / total       0.96      0.96      0.96       445



#### Custom Testing

In [14]:
# Radom instance testing 
import random
def radom_sample_testing(data,model,tfidf_vectorizer,test_cases):
    print model
    sample=random.sample(zip(data.rqst,data.r_type),test_cases)
    new_term_freq_matrix = tfidf_vectorizer.transform([i[0] for i in sample])
    new_term_freq_matrix=new_term_freq_matrix.todense()
    for i in zip(zip([i[0] for i in sample],[i[1] for i in sample]),model.predict(new_term_freq_matrix)):
        print i[0][0]
        print "Actual::\t",i[0][1],"\nPredicted::\t",i[1]

In [15]:
radom_sample_testing(data,model,tfidf_vectorizer,5)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
when did israel begin turning the gaza strip and jericho over to the plo ? 
Actual::	when 
Predicted::	when
what 's the only color johnny cash wears on stage ? 
Actual::	what 
Predicted::	what
what chemicals are used in lethal injection ? 
Actual::	what 
Predicted::	what
who is desmond tutu ? 
Actual::	who 
Predicted::	who
can these be recycled some way ? 
Actual::	affirmation 
Predicted::	affirmation


### User Testing

In [16]:
Query=raw_input()
print  "Predicted::",model.predict(tfidf_vectorizer.transform([Query]).todense())

What time does the train leave?
Predicted:: ['when']


### Support Vector Machine -svm

In [17]:
#Import Library
from sklearn import svm
model = svm.SVC(kernel='linear', C = 1.0)
model.fit(X_train,y_train)
#model.score(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
predicted= model.predict(X_test)
print "Accuracy:",sklearn.metrics.accuracy_score(y_test, predicted)
print "\nConfusion Matrix:\n",sklearn.metrics.confusion_matrix(y_test,predicted)
print "\nclassification Report:\n",sklearn.metrics.classification_report(y_test,predicted)

Accuracy: 0.957303370787

Confusion Matrix:
[[ 25   0   1   0   1]
 [  0  72   3   2   1]
 [  0   1 173   3   2]
 [  0   1   3  24   0]
 [  0   0   0   1 132]]

classification Report:
             precision    recall  f1-score   support

affirmation       1.00      0.93      0.96        27
    unknown       0.97      0.92      0.95        78
       what       0.96      0.97      0.96       179
       when       0.80      0.86      0.83        28
        who       0.97      0.99      0.98       133

avg / total       0.96      0.96      0.96       445



## Random Testing

In [19]:
radom_sample_testing(data,model,tfidf_vectorizer,5)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
who took over as conductor of the boston pops after arthur fiedler 's long reign ? 
Actual::	who 
Predicted::	who
where must a soccer goalie stand to be permitted to handle the ball ? 
Actual::	unknown 
Predicted::	unknown
who wrote the poem that starts `` i love your lips when they 're wet with wine and red with a warm desire '' ? 
Actual::	who 
Predicted::	who
what sprawling u.s. state boasts the most airports ? 
Actual::	what 
Predicted::	what
what is d.b. cooper known for ? 
Actual::	what 
Predicted::	what


### User Testing

In [20]:
Query=raw_input()
print  "Predicted::",model.predict(tfidf_vectorizer.transform([Query]).todense())

What time does the train leave?
Predicted:: ['when']
